# Exploring simplification with [`cityseer`](https://github.com/benchmark-urbanism/cityseer-api)

* **Simons, G.** (2023). *The cityseer Python package for pedestrian-scale network-based urban analysis*. Environment and Planning B: Urban Analytics and City Science, 50(5), 1328-1344. https://doi.org/10.1177/23998083221133827
* Relevant demos in `cityseer`
  * [`graph_cleaning.ipynb`](https://github.com/benchmark-urbanism/cityseer-examples/blob/main/notebooks/graph_cleaning.ipynb)
  * [`graph_corrections.ipynb`](https://github.com/benchmark-urbanism/cityseer-examples/blob/main/notebooks/graph_corrections.ipynb) 

-----------------

1. Setup
2. Prep work
3. Examinging simplification routine
4. Initial vs. Final
5. ...
6. Other stuff
7. ...

-----------------

## 1. Setup

In [1]:
%load_ext watermark
%watermark

Last updated: 2024-03-10T17:37:04.825229-04:00

Python implementation: CPython
Python version       : 3.12.2
IPython version      : 8.22.2

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.4.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit



In [2]:
import pathlib

import cityseer
import geopandas
import momepy
import networkx

from cityseer.tools import graphs, plot, io

%watermark -w
%watermark -iv

Watermark: 2.4.3

cityseer : 4.11.1
networkx : 3.2.1
geopandas: 0.14.3
momepy   : 0.7.1.dev22+g9f98387



In [3]:
ddir = pathlib.Path("..", "data")
parq = "parquet"
samp_parq = ddir / f"sample.{parq}"

In [4]:
sample = geopandas.read_parquet(samp_parq)
sample.head(2)

,eFUA_ID,UC_num,UC_IDs,eFUA_name,Commuting,Cntry_ISO,Cntry_name,FUA_area,UC_area,FUA_p_2015,UC_p_2015,Com_p_2015,geometry,continent,iso_a3
305,9129.0,1.0,8078,Gonda,1.0,IND,India,66.0,29.0,1.074100e+06,1.066419e+06,7680.678101,"POLYGON ((81.98398 27.19657, 81.99471 27.19657...",Asia,IND
91,7578.0,6.0,10577;10581;10583;10596;10605;10607,Chongqing,1.0,CHN,China,2267.0,618.0,6.036834e+06,5.157726e+06,879107.861057,"POLYGON ((106.23972 29.52328, 106.19622 29.523...",Asia,CHN


In [5]:
# dict of fua ID : cityname
fua_city = {
    1133: "Aleppo",
    869: "Auckland",
    4617: "Bucaramanga",
    809: "Douala",
    1656: "Liège",
    4881: "Salt Lake City",
}
# dict of cityname: fua ID
city_fua = {c: f for f, c in fua_city.items()}

In [6]:
def read_parquet_roads(fua: int) -> geopandas.GeoDataFrame:
    """Read OSM roads from parquet format; return bare columns."""
    return (
        geopandas
        .read_parquet(pathlib.Path(ddir, f"{fua}", f"roads_osm.{parq}"))
        [["highway", "geometry"]]
        .reset_index(drop=True)
    )

In [7]:
#city = "Liège"
city = "Douala"
roads_bare = read_parquet_roads(city_fua[city])
roads_bare

,highway,geometry
0,secondary,"LINESTRING (567898.089 460085.416, 567990.584 ..."
1,secondary,"LINESTRING (568040.741 460032.632, 567990.584 ..."
2,secondary,"LINESTRING (567888.251 460090.250, 567898.089 ..."
3,secondary,"LINESTRING (568090.248 460024.822, 568040.741 ..."
4,residential,"LINESTRING (577195.321 445120.540, 577355.012 ..."
...,...,...
88741,residential,"LINESTRING (579972.197 441788.017, 579979.945 ..."
88742,residential,"LINESTRING (579969.509 441803.391, 579972.197 ..."
88743,residential,"LINESTRING (578848.451 446564.664, 578817.331 ..."
88744,residential,"LINESTRING (578849.949 446553.423, 578848.451 ..."


--------------------------------

## 2. Prep work

### Converting to NetworkX object for ingestion into cityseer

In [8]:
def graph_size(info: str, g: networkx.Graph) -> str:    
    return f"{info}\n\t* {g}"

#### Conversion within `momepy`

In [9]:
# G_nx_mm - "networkx Graph process by momepy"
G_nx_mm = momepy.gdf_to_nx(roads_bare, integer_labels=True)

#### Conversion within `cityseer`
* [`io.nx_from_generic_geopandas()`](https://github.com/benchmark-urbanism/cityseer-api/blob/30c9e9bf9078b111a297e3aeab4034465444afee/pysrc/cityseer/tools/io.py#L1010)
* Already performs an initial [`graph.nx_merge_parallel_edges()`](https://github.com/benchmark-urbanism/cityseer-api/blob/30c9e9bf9078b111a297e3aeab4034465444afee/pysrc/cityseer/tools/io.py#L1057)

In [10]:
# G_nx_cs - "networkx Graph process by cityseer"
G_nx_cs = io.nx_from_generic_geopandas(roads_bare)

INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88746/88746 [00:00<00:00, 140391.76it/s]


#### Same elements count? Yes.

In [11]:
print(graph_size(f"NetworkX Graph from Momepy: {city}", G_nx_mm))
print(graph_size(f"NetworkX Graph from Cityseer: {city}", G_nx_cs))

NetworkX Graph from Momepy: Douala
	* MultiGraph with 80052 nodes and 88746 edges
NetworkX Graph from Cityseer: Douala
	* MultiGraph with 80052 nodes and 88746 edges


#### More efficient? Momepy.

In [12]:
%timeit momepy.gdf_to_nx(roads_bare, integer_labels=True)

3.12 s ± 202 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%timeit io.nx_from_generic_geopandas(roads_bare)

INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88746/88746 [00:00<00:00, 267523.92it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88746/88746 [00:00<00:00, 263633.78it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88746/88746 [00:00<00:00, 271794.28it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88746/88746 [00:00<00:00, 284366.65it/s]
INFO:cityseer.tools.graphs:M

8.39 s ± 298 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


* The `momepy.gdf_to_nx()` is clearly more efficient and does not perform hardcoded simplification, like `io.nx_from_generic_geopandas()` does.
* However, the resultant graph objects are not the same. The `momepy.gdf_to_nx()` one does not have the node keys in the format needed next for further prep work.

In [14]:
list(G_nx_mm.nodes)[:3], list(G_nx_mm.edges)[:3]

([0, 1, 2], [(0, 1, 0), (0, 3, 0), (1, 2, 0)])

In [15]:
list(G_nx_cs.nodes)[:3], list(G_nx_cs.edges)[:3]

(['x567898.089-y460085.416',
  'x567990.584-y460046.632',
  'x568040.741-y460032.632'],
 [('x567898.089-y460085.416', 'x567990.584-y460046.632', 0),
  ('x567898.089-y460085.416', 'x567888.251-y460090.25', 0),
  ('x567990.584-y460046.632', 'x568040.741-y460032.632', 0)])

----------------------------------

### Conversion from `networkx` to `cityseer`
* [`io.network_structure_from_nx`](https://github.com/benchmark-urbanism/cityseer-api/blob/30c9e9bf9078b111a297e3aeab4034465444afee/pysrc/cityseer/tools/io.py#L689)

In [16]:
n_gdf, e_gdf, net_struct = io.network_structure_from_nx(
    G_nx_cs, crs=roads_bare.crs.to_epsg()
)

INFO:cityseer.tools.io:Preparing node and edge arrays from networkX graph.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80052/80052 [00:09<00:00, 8022.86it/s]


In [17]:
n_gdf

,ns_node_idx,x,y,live,weight,geom
x567898.089-y460085.416,0,567898.088760,460085.415952,True,1,POINT (567898.089 460085.416)
x567990.584-y460046.632,1,567990.584472,460046.632084,True,1,POINT (567990.584 460046.632)
x568040.741-y460032.632,2,568040.740571,460032.632152,True,1,POINT (568040.741 460032.632)
x567888.251-y460090.25,3,567888.251317,460090.250079,True,1,POINT (567888.251 460090.250)
x568090.248-y460024.822,4,568090.248138,460024.822110,True,1,POINT (568090.248 460024.822)
...,...,...,...,...,...,...
x579972.197-y441788.017,80047,579972.197475,441788.016565,True,1,POINT (579972.197 441788.017)
x579979.945-y441777.079,80048,579979.944806,441777.079476,True,1,POINT (579979.945 441777.079)
x578848.451-y446564.664,80049,578848.451451,446564.663950,True,1,POINT (578848.451 446564.664)
x578817.331-y446570.551,80050,578817.330514,446570.550882,True,1,POINT (578817.331 446570.551)


In [18]:
e_gdf

,ns_edge_idx,start_ns_node_idx,end_ns_node_idx,edge_idx,nx_start_node_key,nx_end_node_key,length,angle_sum,imp_factor,in_bearing,out_bearing,total_bearing,geom
x567898.089-y460085.416-x567990.584-y460046.632,0,0,1,0,x567898.089-y460085.416,x567990.584-y460046.632,100.297783,0,1,-22.748523,-22.748523,-22.748523,"LINESTRING (567898.089 460085.416, 567990.584 ..."
x567898.089-y460085.416-x567888.251-y460090.25,1,0,3,0,x567898.089-y460085.416,x567888.251-y460090.25,10.961024,0,1,153.830465,153.830465,153.830465,"LINESTRING (567888.251 460090.250, 567898.089 ..."
x567990.584-y460046.632-x567898.089-y460085.416,2,1,0,0,x567990.584-y460046.632,x567898.089-y460085.416,100.297783,0,1,157.251477,157.251477,157.251477,"LINESTRING (567898.089 460085.416, 567990.584 ..."
x567990.584-y460046.632-x568040.741-y460032.632,3,1,2,0,x567990.584-y460046.632,x568040.741-y460032.632,52.073336,0,1,-15.595864,-15.595864,-15.595864,"LINESTRING (568040.741 460032.632, 567990.584 ..."
x568040.741-y460032.632-x567990.584-y460046.632,4,2,1,0,x568040.741-y460032.632,x567990.584-y460046.632,52.073336,0,1,164.404136,164.404136,164.404136,"LINESTRING (568040.741 460032.632, 567990.584 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
x578848.451-y446564.664-x578817.331-y446570.551,177487,80049,80050,0,x578848.451-y446564.664,x578817.331-y446570.551,31.672838,0,1,169.288322,169.288322,169.288322,"LINESTRING (578848.451 446564.664, 578817.331 ..."
x578848.451-y446564.664-x578849.949-y446553.423,177488,80049,80051,0,x578848.451-y446564.664,x578849.949-y446553.423,11.340314,0,1,-82.412697,-82.412697,-82.412697,"LINESTRING (578849.949 446553.423, 578848.451 ..."
x578817.331-y446570.551-x578848.451-y446564.664,177489,80050,80049,0,x578817.331-y446570.551,x578848.451-y446564.664,31.672838,0,1,-10.711678,-10.711678,-10.711678,"LINESTRING (578848.451 446564.664, 578817.331 ..."
x578849.949-y446553.423-x578848.035-y446533.457,177490,80051,79164,0,x578849.949-y446553.423,x578848.035-y446533.457,20.057480,0,1,-95.476160,-95.476160,-95.476160,"LINESTRING (578848.035 446533.457, 578849.949 ..."


In [19]:
net_struct

------------------------------------

## 3. Examining the simplification routine

### Question:

Is there a single, 'simplify' routine?

### Answer

Sort of. but from what I can tell it's only available as a keyword argument in [`cityseer.tools.io.osm_graph_from_poly()`](https://github.com/benchmark-urbanism/cityseer-api/blob/30c9e9bf9078b111a297e3aeab4034465444afee/pysrc/cityseer/tools/io.py#L218). The procedure is basically as follows:

1. `graphs.nx_simple_geoms()` – happens even if `simplify=False`
2. `graphs.nx_remove_filler_nodes()` – happens even if `simplify=False`
3.  `graphs.nx_remove_dangling_nodes()`
4.  `graphs.nx_consolidate_nodes(crawl=True)`
    1. `graphs.nx_remove_filler_nodes`
    1. `graphs.nx_merge_parallel_edges()`
5.   `graphs.nx_split_opposing_geoms()`
     1.  `graphs.nx_merge_parallel_edges()`
6.  `graphs.nx_consolidate_nodes(crawl=False)`
     1. `graphs.nx_remove_filler_nodes()` 
7.  `graphs.nx_remove_filler_nodes()`
8.  `graphs.nx_iron_edges()`
     1.  `graphs.nx_merge_parallel_edges()`
9.  `graphs.nx_split_opposing_geoms()`
     1.  `graphs.nx_merge_parallel_edges()`
10.  `graphs.nx_consolidate_nodes(crawl=False)`
     1. `graphs.nx_remove_filler_nodes()` 
11.  `graphs.nx_remove_filler_nodes()`
12.  `graphs.nx_iron_edges()`
     1. `graphs.nx_merge_parallel_edges()`

### Default params of `osm_graph_from_poly()` relating to simplification

* `crawl_consolidate_dist: int = 12`
* `parallel_consolidate_dist: int = 15`
* `contains_buffer_dist: int = 50`
* `iron_edges: bool = True`
* `remove_disconnected: int = 100`

### Breaking down the routine
#### Setting params (based on `io.osm_graph_from_poly()`

In [20]:
crawl_consolidate_dist: int = 12
parallel_consolidate_dist: int = 15
contains_buffer_dist: int = 50
iron_edges: bool = True
remove_disconnected: int = 100

#### 3.1. – `graphs.nx_simple_geoms()`
* parameters – see [here](https://github.com/benchmark-urbanism/cityseer-api/blob/30c9e9bf9078b111a297e3aeab4034465444afee/pysrc/cityseer/tools/graphs.py#L29)
  * none
* called [here](https://github.com/benchmark-urbanism/cityseer-api/blob/850d451886532d9dfd204c3be609ef7136746d2c/pysrc/cityseer/tools/io.py#L356)

In [21]:
# smp - "simple geoms"
G_nx_mm_smp = graphs.nx_simple_geoms(G_nx_mm)
nxmp = "NetworkX-Momepy"
print(graph_size(f"{nxmp}: `nx_simple_geoms`: {city}", G_nx_mm_smp))

INFO:cityseer.tools.graphs:Generating interpolated edge geometries.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88746/88746 [00:01<00:00, 82466.20it/s]


NetworkX-Momepy: `nx_simple_geoms`: Douala
	* MultiGraph with 80052 nodes and 88746 edges


* Seemingling unneeded step as it doesn't decrease element count. However...
* ... must be run to perform the next step – `graphs.nx_remove_filler_nodes()`
  * (can't pass in the Momepy generated object) 

In [22]:
graphs.nx_remove_filler_nodes(G_nx_mm)

INFO:cityseer.tools.graphs:Removing filler nodes.
  0%|                                                                                                                                             | 0/80052 [00:00<?, ?it/s]


KeyError: 'Missing "geom" attribute for edge 4-2'

#### 3.2. – `graphs.nx_remove_filler_nodes()`
* parameters – see [here](https://github.com/benchmark-urbanism/cityseer-api/blob/30c9e9bf9078b111a297e3aeab4034465444afee/pysrc/cityseer/tools/graphs.py#L86C5-L86C27)
  * none
* called [here](https://github.com/benchmark-urbanism/cityseer-api/blob/850d451886532d9dfd204c3be609ef7136746d2c/pysrc/cityseer/tools/io.py#L357)

In [23]:
# rfn1st - "remove filler nodes - 1st call"
G_nx_cs_rfn1st = graphs.nx_remove_filler_nodes(G_nx_mm_smp)
print(graph_size(f"{nxmp}: `nx_remove_filler_nodes - 1`: {city}", G_nx_cs_rfn1st))

INFO:cityseer.tools.graphs:Removing filler nodes.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80052/80052 [00:09<00:00, 8168.91it/s]

NetworkX-Momepy: `nx_remove_filler_nodes - 1`: Douala
	* MultiGraph with 23326 nodes and 32020 edges


* Significant decrease in elements.

#### 3.3. – `graphs.nx_remove_dangling_nodes()`
* parameters – see [here](https://github.com/benchmark-urbanism/cityseer-api/blob/30c9e9bf9078b111a297e3aeab4034465444afee/pysrc/cityseer/tools/graphs.py#L235)
  * `despine: int = 15`
  * `remove_disconnected: int = remove_disconnected` --> 100
  * `cleanup_filler_nodes: bool = True`
* called [here](https://github.com/benchmark-urbanism/cityseer-api/blob/850d451886532d9dfd204c3be609ef7136746d2c/pysrc/cityseer/tools/io.py#L359)
* func(s) called within:
  * `nx_remove_filler_nodes()` – 2nd call

In [24]:
# rdn - "remove dangling nodes"
G_nx_cs_rdn = graphs.nx_remove_dangling_nodes(
    G_nx_cs_rfn1st,
    despine=15,
    remove_disconnected=remove_disconnected,
    cleanup_filler_nodes=True,
)
print(graph_size(f"{nxmp}: `nx_remove_dangling_nodes`: {city}", G_nx_cs_rdn))

INFO:cityseer.tools.graphs:Removing dangling nodes.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23187/23187 [00:00<00:00, 267488.11it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23149/23149 [00:00<00:00, 526985.65it/s]

NetworkX-Momepy: `nx_remove_dangling_nodes`: Douala
	* MultiGraph with 23115 nodes and 31814 edges


#### 3.4. – `graphs.nx_consolidate_nodes(crawl=True, buffer_dist=12)`  - 1st call
* parameters – see [here](https://github.com/benchmark-urbanism/cityseer-api/blob/850d451886532d9dfd204c3be609ef7136746d2c/pysrc/cityseer/tools/graphs.py#L656)
  * `buffer_dist: float = crawl_consolidate_dist` --> 12
  * `neighbour_policy: str | None = None`
  * `crawl: bool = True`
  * `centroid_by_itx: bool = True`
  * `merge_edges_by_midline: bool = True`
  * `contains_buffer_dist: int = contains_buffer_dist` --> 50
* called [here](https://github.com/benchmark-urbanism/cityseer-api/blob/850d451886532d9dfd204c3be609ef7136746d2c/pysrc/cityseer/tools/io.py#L360) 
* func(s) called within:
  * `graphs.nx_remove_filler_nodes()` – 3rd call
  * ***>>>> `graphs.nx_merge_parallel_edges()` – 1st call <<<<***

In [25]:
# cn1st - "consolidate nodes - 1st call"
G_nx_cs_cn1st = graphs.nx_consolidate_nodes(
    G_nx_cs_rdn,
    buffer_dist=crawl_consolidate_dist,
    neighbour_policy=None,
    crawl=True,
    centroid_by_itx=True,
    merge_edges_by_midline=True,
    contains_buffer_dist=contains_buffer_dist,
)
print(graph_size(f"{nxmp}: `nx_consolidate_nodes` - 1: {city}", G_nx_cs_cn1st))

INFO:cityseer.tools.util:Creating nodes STR tree
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23115/23115 [00:00<00:00, 125786.35it/s]
INFO:cityseer.tools.graphs:Consolidating nodes.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23115/23115 [00:02<00:00, 8353.80it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21817/21817 [00:00<00:00, 504892.00it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 50.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30436/30436 [00:00<00:00, 98322.45it/s]

NetworkX-Momepy: `nx_consolidate_nodes` - 1: Douala
	* MultiGraph with 21790 nodes and 30262 edges


#### 3.5. – `graphs.nx_split_opposing_geoms()` - 1st call
* parameters – see [here](https://github.com/benchmark-urbanism/cityseer-api/blob/850d451886532d9dfd204c3be609ef7136746d2c/pysrc/cityseer/tools/graphs.py#L809)
  * `buffer_dist: float = parallel_consolidate_dist` --> 15
  * `merge_edges_by_midline: bool = True`
  * `contains_buffer_dist: int = contains_buffer_dist` --> 50
* called [here](https://github.com/benchmark-urbanism/cityseer-api/blob/850d451886532d9dfd204c3be609ef7136746d2c/pysrc/cityseer/tools/io.py#L366)
* func(s) called within:
  * ***>>>> `graphs.nx_merge_parallel_edges()` – 2nd call <<<<***

In [26]:
# sog1st - "split opposing geoms - 1st call"
G_nx_cs_sog1st = graphs.nx_split_opposing_geoms(
    G_nx_cs_cn1st,
    buffer_dist=parallel_consolidate_dist,
    merge_edges_by_midline=True,
    contains_buffer_dist=contains_buffer_dist
)
print(graph_size(f"{nxmp}: `nx_split_opposing_geoms` - 1: {city}", G_nx_cs_sog1st))

INFO:cityseer.tools.util:Creating edges STR tree.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30262/30262 [00:00<00:00, 680517.21it/s]
INFO:cityseer.tools.graphs:Splitting opposing edges.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21790/21790 [00:04<00:00, 4687.38it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 50.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30728/30728 [00:00<00:00, 211183.20it/s]

NetworkX-Momepy: `nx_split_opposing_geoms` - 1: Douala
	* MultiGraph with 22256 nodes and 30728 edges


#### 3.6. – `graphs.nx_consolidate_nodes(crawl=False, buffer_dist=15)`  - 2nd call
* parameters
  * `buffer_dist: float = parallel_consolidate_dist` --> 15
  * `neighbour_policy: str | None = None`
  * `crawl: bool = False`
  * `centroid_by_itx: bool = True`
  * `merge_edges_by_midline: bool = True`
  * `contains_buffer_dist: int = contains_buffer_dist` --> 50
* called [here](https://github.com/benchmark-urbanism/cityseer-api/blob/850d451886532d9dfd204c3be609ef7136746d2c/pysrc/cityseer/tools/io.py#L369)
* func(s) called within:
  * `graphs.nx_remove_filler_nodes()` – 3rd call
  * ***>>>> `graphs.nx_merge_parallel_edges()` – 2nd call <<<<***

In [27]:
# cn2nd - "consolidate nodes - 2nd call"
G_nx_cs_cn2nd = graphs.nx_consolidate_nodes(
    G_nx_cs_sog1st,
    buffer_dist=parallel_consolidate_dist,
    neighbour_policy=None,
    crawl=False,
    centroid_by_itx=True,
    merge_edges_by_midline=True,
    contains_buffer_dist=contains_buffer_dist,
)
print(graph_size(f"{nxmp}: `nx_consolidate_nodes` - 2: {city}", G_nx_cs_cn2nd))

INFO:cityseer.tools.util:Creating nodes STR tree
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22256/22256 [00:00<00:00, 122219.96it/s]
INFO:cityseer.tools.graphs:Consolidating nodes.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22256/22256 [00:02<00:00, 10046.26it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21456/21456 [00:00<00:00, 308687.79it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 50.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30125/30125 [00:00<00:00, 65254.81it/s]

NetworkX-Momepy: `nx_consolidate_nodes` - 2: Douala
	* MultiGraph with 21328 nodes and 29709 edges


#### 3.7. – `nx_remove_filler_nodes` – 4th call
* parameters – see above
  * none
* called [here](https://github.com/benchmark-urbanism/cityseer-api/blob/850d451886532d9dfd204c3be609ef7136746d2c/pysrc/cityseer/tools/io.py#L374)

In [28]:
# rfn4th - "remove filler nodes - 4th call"
G_nx_cs_rfn4th = graphs.nx_remove_filler_nodes(G_nx_cs_cn2nd)
print(graph_size(f"{nxmp}: `nx_remove_filler_nodes - 4`: {city}", G_nx_cs_rfn4th))

INFO:cityseer.tools.graphs:Removing filler nodes.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21328/21328 [00:00<00:00, 373526.00it/s]

NetworkX-Momepy: `nx_remove_filler_nodes - 4`: Douala
	* MultiGraph with 21272 nodes and 29653 edges


#### 3.8. – `graphs.nx_iron_edges()` – 1st call
* parameters – see [here](https://github.com/benchmark-urbanism/cityseer-api/blob/850d451886532d9dfd204c3be609ef7136746d2c/pysrc/cityseer/tools/graphs.py#L452)
  * none
* called [here](https://github.com/benchmark-urbanism/cityseer-api/blob/850d451886532d9dfd204c3be609ef7136746d2c/pysrc/cityseer/tools/io.py#L376)
* func(s) called within:
  * ***>>>> `graphs.nx_merge_parallel_edges()` – 3rd call <<<<***

In [29]:
# ie1st - "iron edges - 1st call"
G_nx_cs_ie1st = graphs.nx_iron_edges(G_nx_cs_rfn4th)
print(graph_size(f"{nxmp}: `nx_iron_edges - 1`: {city}", G_nx_cs_ie1st))

INFO:cityseer.tools.graphs:Ironing edges.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29653/29653 [00:06<00:00, 4617.51it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29653/29653 [00:00<00:00, 222991.44it/s]

NetworkX-Momepy: `nx_iron_edges - 1`: Douala
	* MultiGraph with 21272 nodes and 29653 edges


#### 3.9. – `graphs.nx_split_opposing_geoms()` - 2nd call
* parameters – see [here](https://github.com/benchmark-urbanism/cityseer-api/blob/850d451886532d9dfd204c3be609ef7136746d2c/pysrc/cityseer/tools/graphs.py#L809)
  * `buffer_dist: float = parallel_consolidate_dist` --> 15
  * `merge_edges_by_midline: bool = True`
  * `contains_buffer_dist: int = contains_buffer_dist` --> 50
* called [here](https://github.com/benchmark-urbanism/cityseer-api/blob/850d451886532d9dfd204c3be609ef7136746d2c/pysrc/cityseer/tools/io.py#L377)
* func(s) called within:
  * ***>>>> `graphs.nx_merge_parallel_edges()` – 3rd call <<<<*** 

In [30]:
# sog2nd - "split opposing geoms - 2nd call"
G_nx_cs_sog2nd = graphs.nx_split_opposing_geoms(
    G_nx_cs_ie1st,
    buffer_dist=parallel_consolidate_dist,
    merge_edges_by_midline=True,
    contains_buffer_dist=contains_buffer_dist
)
print(graph_size(f"{nxmp}: `nx_split_opposing_geoms` - 2: {city}", G_nx_cs_sog2nd))

INFO:cityseer.tools.util:Creating edges STR tree.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29653/29653 [00:00<00:00, 747244.98it/s]
INFO:cityseer.tools.graphs:Splitting opposing edges.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21272/21272 [00:03<00:00, 6457.57it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 50.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29734/29734 [00:00<00:00, 221378.25it/s]

NetworkX-Momepy: `nx_split_opposing_geoms` - 2: Douala
	* MultiGraph with 21353 nodes and 29732 edges


#### 3.10. – `graphs.nx_consolidate_nodes(crawl=False, buffer_dist=15)`  - 3rd call
* parameters
  * `buffer_dist: float = parallel_consolidate_dist` --> 15
  * `neighbour_policy: str | None = None`
  * `crawl: bool = False`
  * `centroid_by_itx: bool = True`
  * `merge_edges_by_midline: bool = True`
  * `contains_buffer_dist: int = contains_buffer_dist` --> 50
* called [here](https://github.com/benchmark-urbanism/cityseer-api/blob/850d451886532d9dfd204c3be609ef7136746d2c/pysrc/cityseer/tools/io.py#L380)
* func(s) called within:
  * `graphs.nx_remove_filler_nodes()` – 5th call
  * ***>>>> `graphs.nx_merge_parallel_edges()` – 4th call <<<<***

In [31]:
# cn3rd - "consolidate nodes - 3rd call"
G_nx_cs_cn3rd = graphs.nx_consolidate_nodes(
    G_nx_cs_sog2nd,
    buffer_dist=parallel_consolidate_dist,
    neighbour_policy=None,
    crawl=False,
    centroid_by_itx=True,
    merge_edges_by_midline=True,
    contains_buffer_dist=contains_buffer_dist,
)
print(graph_size(f"{nxmp}: `nx_consolidate_nodes` - 3: {city}", G_nx_cs_cn3rd))

INFO:cityseer.tools.util:Creating nodes STR tree
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21353/21353 [00:00<00:00, 111830.44it/s]
INFO:cityseer.tools.graphs:Consolidating nodes.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21353/21353 [00:01<00:00, 14396.51it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21256/21256 [00:00<00:00, 680020.79it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 50.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29670/29670 [00:00<00:00, 32433.49it/s]

NetworkX-Momepy: `nx_consolidate_nodes` - 3: Douala
	* MultiGraph with 21250 nodes and 29590 edges


#### 3.11. – `nx_remove_filler_nodes` – 5th call
* parameters – see above
  * none
* called [here](https://github.com/benchmark-urbanism/cityseer-api/blob/850d451886532d9dfd204c3be609ef7136746d2c/pysrc/cityseer/tools/io.py#L385)

In [32]:
# rfn6th - "remove filler nodes - 6th call"
G_nx_cs_rfn6th = graphs.nx_remove_filler_nodes(G_nx_cs_cn3rd)
print(graph_size(f"{nxmp}: `nx_remove_filler_nodes - 6`: {city}", G_nx_cs_rfn6th))

INFO:cityseer.tools.graphs:Removing filler nodes.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21250/21250 [00:00<00:00, 682933.44it/s]

NetworkX-Momepy: `nx_remove_filler_nodes - 6`: Douala
	* MultiGraph with 21243 nodes and 29583 edges


#### 3.12. – `graphs.nx_iron_edges()` – 2nd call
* parameters – see above
  * none
* called [here](https://github.com/benchmark-urbanism/cityseer-api/blob/850d451886532d9dfd204c3be609ef7136746d2c/pysrc/cityseer/tools/io.py#L387)
* func(s) called within:
  * ***>>>> `graphs.nx_merge_parallel_edges()` – 5th call <<<<***

In [33]:
# ie2nd - "iron edges - 2nd call"
G_nx_cs_ie2nd = graphs.nx_iron_edges(G_nx_cs_rfn6th)
print(graph_size(f"{nxmp}: `nx_iron_edges - 2`: {city}", G_nx_cs_ie2nd))

INFO:cityseer.tools.graphs:Ironing edges.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29583/29583 [00:05<00:00, 5151.38it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29583/29583 [00:00<00:00, 229732.01it/s]

NetworkX-Momepy: `nx_iron_edges - 2`: Douala
	* MultiGraph with 21243 nodes and 29583 edges


------------------------------
## 4. Initial vs. Final

In [34]:
print(
    f"Started with {graph_size(f'raw: {city}', G_nx_cs)}\n"
    f"Ended with – {graph_size(f'simplified: {city}', G_nx_cs_ie2nd)}"
)

Started with raw: Douala
	* MultiGraph with 80052 nodes and 88746 edges
Ended with – simplified: Douala
	* MultiGraph with 21243 nodes and 29583 edges


### Prep for some viz

In [37]:
fpath = pathlib.Path("..", "usecases", "809", "cityseer", "outdata")
fbase = "{}_{}_{}.geojson"

##### Raw/bare network

In [38]:
mm_nodes_raw, mm_edges_raw = momepy.nx_to_gdf(G_nx_mm)
(
    mm_nodes_raw
    .to_crs(4326)
    .to_file(fpath / fbase.format("mm", "nodes", "raw"))
)
(
    mm_edges_raw
    .to_crs(4326)
    .to_file(fpath / fbase.format("mm", "edges", "raw"))
)

##### Simplified network

In [39]:
cs_road_simp = (
    io
    .geopandas_from_nx(G_nx_cs_ie2nd, crs=roads_bare.crs)
    .rename_geometry("geometry")
)
cs_G_simp = momepy.gdf_to_nx(cs_road_simp, integer_labels=True)
cs_nodes_simp, cs_edges_simp = momepy.nx_to_gdf(cs_G_simp)
(
    cs_nodes_simp
    .to_crs(4326)
    .to_file(fpath / fbase.format("cs", "nodes", "simplified"))
)
(
    cs_edges_simp
    .to_crs(4326)
    .to_file(fpath / fbase.format("cs", "edges", "simplified"))
)

INFO:cityseer.tools.io:Preparing node and edge arrays from networkX graph.


In [40]:
cs_nodes_simp, cs_edges_simp = momepy.nx_to_gdf(cs_G_simp)
cs_nodes_simp.to_file(fpath / fbase.format("cs", "nodes", "simplified"))
cs_edges_simp.to_file(fpath / fbase.format("cs", "edges", "simplified"))

-------------------------------

## 5. Other stuff

* `graphs.nx_weight_by_dissolved_edges()`
  * see [here](https://github.com/benchmark-urbanism/cityseer-api/blob/850d451886532d9dfd204c3be609ef7136746d2c/pysrc/cityseer/tools/graphs.py#L1345)

-----------------------------------